# 强化学习 Q learning算法
这一次我们会用 tabular Q-learning 的方法实现一个小例子, 例子的环境是一个一维世界, 在世界的右边有宝藏, 探索者只要得到宝藏尝到了甜头, 然后以后就记住了得到宝藏的方法, 这就是他用强化学习所学习到的行为.

-o---T

#T 就是宝藏的位置, o 是探索者的位置
Q-learning 是一种记录行为值 (Q value) 的方法, 每种在一定状态的行为都会有一个值 Q(s, a), 就是说 行为 a 在 s 状态的值是 Q(s, a). s 在上面的探索者游戏中, 就是 o 所在的地点了. 而每一个地点探索者都能做出两个行为 left/right, 这就是探索者的所有可行的 a 啦.

如果在某个地点 s1, 探索者计算了他能有的两个行为, a1/a2=left/right, 计算结果是 Q(s1, a1) > Q(s1, a2), 那么探索者就会选择 left 这个行为. 这就是 Q learning 的行为选择简单规则.

In [1]:
import numpy as np
import pandas as pd
import time

np.random.seed(2)

N_STATES = 7     #一维世界的长度
ACTIONS = ['left','right'] #可采取的行动
EPSILON = 0.9   #贪婪度
ALPHA = 0.1    #learning rate
GAMMA = 0.9     #奖励递减值
MAX_EPISODES = 13 #最大回合数
FRESH_TIME = 0.3  #移动间隔时间


# Q表
对于 tabular Q learning, 我们必须将所有的 Q values (行为值) 放在 q_table 中, 更新 q_table 也是在更新他的行为准则. q_table 的 index 是所有对应的 state (探索者位置), columns 是对应的 action (探索者行为).

In [2]:
def build_q_table(n_states,actions):
    table = pd.DataFrame(
          np.zeros((n_states,len(actions))),#q_table全0初始化
          columns = actions,  # columns对应行为名称
    )
    return table

 # 定义动作
 接着定义探索者是如何挑选行为的. 这是我们引入 epsilon greedy 的概念. 因为在初始阶段, 随机的探索环境, 往往比固定的行为模式要好, 所以这也是累积经验的阶段, 我们希望探索者不会那么贪婪(greedy). 所以 EPSILON 就是用来控制贪婪程度的值. EPSILON 可以随着探索时间不断提升(越来越贪婪), 不过在这个例子中, 我们就固定成 EPSILON = 0.9, 90% 的时间是选择最优策略, 10% 的时间来探索.

In [3]:
def choose_action(state,q_table):
    state_actions = q_table.iloc[state,:] # 选出这个 state 的所有 action 值
    if(np.random.uniform()>EPSILON)or(state_actions.all()==0): # 非贪婪 or 或者这个 state 还没有探索过
        action_name =  np.random.choice(ACTIONS)
    else:
        action_name = state_actions.argmax()#贪婪模式
    return action_name

# 环境反馈
做出行为后, 环境也要给我们的行为一个反馈, 反馈出下个 state (S_) 和 在上个 state (S) 做出 action (A) 所得到的 reward (R). 这里定义的规则就是, 只有当 o 移动到了 T, 探索者才会得到唯一的一个奖励, 奖励值 R=1, 其他情况都没有奖励.

In [4]:
def get_env_feedback(S,A):
    if A == 'right':
        if S==N_STATES-2: #terminate
            S_ = 'terminal'
            R=1
        else:
            S_=S+1
            R=0
    else:
        R=0
        if S==0:
            S_=S
        else:
            S_=S-1
    return S_,R

In [5]:
#环境更新
def update_env(S,episode,step_counter):
    env_list = ['-']*(N_STATES-1)+['T']
    if S == 'terminal':
        interaction = 'Episode %s:total_steps=%s'%(episode+1,step_counter)
        print('\r{}'.format(interaction),end='')
        time.sleep(2)
        print('\r         ',end='')
    else:
        env_list[S]='o'
        interaction = ''.join(env_list)
        print('\r{}'.format(interaction),end='')
        time.sleep(FRESH_TIME)

# 强化学习主循环


In [6]:
def rl():
    q_table = build_q_table(N_STATES,ACTIONS) #初始化q table
    for episode in range(MAX_EPISODES):
        step_counter = 0
        S = 0
        is_terminated = False  #是否回合结束
        update_env(S,episode,step_counter) #更新环境
        while not is_terminated:
            A = choose_action(S,q_table) #选择行为
            S_,R = get_env_feedback(S,A)
            q_predict = q_table.loc[S,A]
            if S_!='terminal':
                q_target = R+GAMMA*q_table.iloc[S_,:].max()  #  实际的(状态-行为)值 (回合没结束)
            
            else:
                q_target = R
                is_terminated = True
            q_table.loc[S,A]+=ALPHA*(q_target-q_predict) #q_table跟新
            S=S_ #探索者移动到下一个state
            update_env(S,episode,step_counter+1)
            
            step_counter +=1
        return q_table

In [7]:
if __name__ == "__main__":
    q_table = rl()
    print('\r\nQ-table:\n')
    print(q_table)

         :total_steps=45
Q-table:

   left  right
0   0.0    0.0
1   0.0    0.0
2   0.0    0.0
3   0.0    0.0
4   0.0    0.0
5   0.0    0.1
6   0.0    0.0
